
# 필요 라이브러리 임포트


In [ ]:
from pycocotools.coco import COCO
import torch
import torch.nn as nn
import matplotlib
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import pprint
import json
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import copy

# get_classname : 클래스 아이디로 부터 이름을 얻어오는 코드
# visualize_bbox : 각 어노테이션으로 부터 bbox를 그려준다.
# visualize : bbox를 포함한 이미지를 출력하는 코드



In [ ]:
def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

category_names = ['Backgroud','UNKNOWN','General trash','Paper','Paper pack','Metal','Glass','Plastic','Styrofoam','Plastic bag','Battery','Clothing']
dataset_path = './data'
anns_file_path = dataset_path + '/' + 'train_all.json'
anns_file_path

BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)


# 데이터 검증용 코드
*   coco데이터 셋을 생성완료한후 image_id annotation_id 가 연속인지 체크해주는 코드 블럭
*   기존데이터로 부터 커스텀 데이터를 만들때 id들이 연속으로 되어있지 않으면 오류 발생 
*   사람마다 데이터셋을 만드는게 다른데 enumerate를 이용해서 인덱스를 id로 사용하는 경우가 있음
*    코드상 id를 연속으로 만들도록 하고 다시한번 체크해준다


In [ ]:
ntap = "./data/new_train_all.json"
with open(ntap,'r') as json_file:
  json_data = json.load(json_file)

for i,info in enumerate(json_data["images"]):
  if(i == (len(json_data["images"])-1)):
    break
  id1 = json_data["images"][i]['id']
  id2 = json_data["images"][i+1]['id']

  if((id2-id1) != 1):
    print("!!!!!")

print(json_data["images"][0]['id'])
print(json_data["images"][len(json_data["images"])-1]['id'])
print(len(json_data["images"]))

# 이미지와 coco데이터 셋에서 부족한 오브젝트를 뜯어온다.


In [ ]:
class CoCoDataAugment(Dataset):
    """COCO format"""

    def __init__(self, data_dir):
        super().__init__()
        self.coco = COCO(data_dir)
        self.cat_ids = self.coco.getCatIds()    
        self.cats = self.coco.loadCats(self.cat_ids)
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]

        # cv2 를 활용하여 image 불러오기
        image = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        # 이미지에 대한 annotation
        anns = self.read_ann_in_img(image_infos['id'])
            
        # mask : size가 (height x width)인 2D
        # 각각의 pixel 값에는 "category id + 1" 할당
        # Unknown = 1, General trash = 2, ... , Cigarette = 11
        count = 0
        bboxes = []
        #print(anns[0])

        images = []
        images_a = []
        masks = []
        bbox =[]
        areas =[]
        labels = []
        is_crowds = []
        for i in range(len(anns)):
            # 클래스의 이름을 얻어 온다.
            className = get_classname(anns[i]['category_id'], self.cats)
            # 마스크에 저장할 값을 얻어온다.
            pixel_value = category_names.index(className)
            # 이미지 크기만한 빈 이미지를 만들고 어노테이션 마다 마스트를 만들어 준다.
            mask = np.zeros((image_infos["height"], image_infos["width"]))
            mask = np.maximum(self.coco.annToMask(anns[i]) * pixel_value, mask)
            mask = mask.astype(np.float32)
            # bbox를 가져온다.
            box = anns[i]['bbox']
            # 이미지와 마스크 이미지에서 bbox만큼 크롭한다.
            crop_img, crop_mask = self.crop_with_bbox(image, mask, box)
            # 마스크를 이용해서 이미지에서 오브젝트만 남기고 지운다. 파일에서 보기 좋게 알파값도 만들어 온다.           
            crop_img, crop_img_a = self.rm_bg_from_img(crop_img, crop_mask)

            # 각 anno마나 이미지, 알파값, 마스크를 저장한다 나머지 annotation을 구성할 정보도 가져온다.
            images.append(crop_img)
            images_a.append(crop_img_a)
            masks.append(crop_mask)
            areas.append(anns[i]['area'])
            labels.append(anns[i]['category_id'])
            is_crowds.append(anns[i]['iscrowd'])

        # 리턴
        return images, images_a, masks, areas, labels, is_crowds

    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

    # 이미지와 마스크 이미지에서 bbox만큼 크롭한다.
    def crop_with_bbox(self, image, mask, bbox):
        x_min = bbox[0]
        y_min = bbox[1]
        x_max = bbox[0] + bbox[2]
        y_max = bbox[1] + bbox[3]
        transform = A.Compose([
                    A.Crop(int(x_min),int(y_min),int(x_max),int(y_max)),
                    ToTensorV2()
            ])
        transformed = transform(image=image, mask=mask)
        crop_img = transformed["image"]
        #print("crop_img type: ",type(crop_img))
        crop_masks = transformed["mask"]
        #print("crop_masks type: ",type(crop_masks))
        return crop_img, crop_masks

    # 마스크를 이용해서 이미지에서 오브젝트만 남기고 지운다. 파일에서 보기 좋게 알파값도 만들어 온다.
    def rm_bg_from_img(self, image, mask):
        bit_mask_base = torch.ones_like(image).to(torch.int8)
        alpha = torch.ones_like(mask).to(torch.float)

        for i in list(torch.unique(mask)):
          if i == 0:
            continue
          bit_mask = bit_mask_base * i
          bit_mask = torch.eq(mask, bit_mask)                    
          temp_image =  image * bit_mask
          temp_alpha = alpha * bit_mask[0]
        return  temp_image, temp_alpha

    def read_ann_in_img(self, img_id):
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        return anns

# 뜯어온 이미지를 일단 annotation과 함께 저장

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))


#  class CoCoDataAugment(Dataset) 으로부터 각이미지에 대해서 생성된 분리된 객체들을 coco데이터 셋으로 만들기위해서 각 객체,마스크,annotation을 저장한다.
def Get_Anno_instance_2(data_dir):
    # CoCoDataAugment와 데이터 로더로 이미지에서 오브젝트를 떼어온다.
    train_all_path = anns_file_path
    train_dataset = CoCoDataAugment(data_dir=train_all_path)
    data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=1,
                                           shuffle=True,
                                           num_workers=4,
                                           collate_fn=collate_fn,
                                           drop_last=True)
    
    count = 0
    json_list = []
    for step,(images, images_a, masks, areas, labels, is_crowds) in tqdm(enumerate(data_loader)):
      #각 오브젝트 이미지당 아래를 수행
      for j in range(0,len(images)):
        for k in range(0,len(images[j])):
          # 내가 원하는 오브젝트가 아니면 무시한다.
          if (labels[j][k] != 3):
            continue
          # 내가 원하는 오브젝트면 
          # 마스크랑 이미지의 파일이름을 만든다.
          save_filename = f'{count:0{9}}' # e.g. 000000023
          img_filename = f'{save_filename}.png'
          mask_filename = f'{save_filename}_m.jpg'
          img_filename = os.path.join(data_dir,img_filename)
          mask_filename = os.path.join(data_dir,mask_filename)

          #검사용으로 이미지 한번 출력
          # fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 12))
          # print("images len: ",len(images))
          # print("images type: ",type(images))
          # print("images[j] len: ",len(images[j][0]))
          # print("images[j] type: ",type(images[j][0]))
          # ax1.imshow(images[j][k].permute([1, 2, 0]))
          # ax1.grid(False)
          # ax2.imshow(masks[j][k])
          # ax2.grid(False)
          # plt.show()

          # opencv를 이용해서 이미지를 저장 bgr로 저장해야함. 마스크 값은 0~10은 너무 값이 작아서 적당히 25를 곱해서 이미지로 봤을때 마스크가 잘 보이도록 한다.
          bgr_img = images[j][k].permute([1, 2, 0]).to('cpu').numpy()
          bgr_img *= 255.0
          b, g, r = cv2.split(bgr_img)

          img_a = images_a[j][k].to('cpu').numpy()
          img_a *= 225.0

          mask_img = masks[j][k].to('cpu').numpy()
          mask_img *= 25
          
          bgra_img = cv2.merge((r, g, b, img_a))

          cv2.imwrite(img_filename, bgra_img)
          cv2.imwrite(mask_filename, mask_img)
          # 오브젝트를 추가할때 사용할 annotation데이터를 파일이름과 함께 저장.
          json_dict = { 'Image_name': img_filename, 'mask_name': mask_filename, 'area':areas[j][k], 'label': labels[j][k], 'is_crowd':is_crowds[j][k]}
          json_list.append(json_dict)


          count += 1
    #각 클래스에 대해서 이미지와 json형식으로 정보 저장. 
    with open(data_dir + "/stencil.json", "w") as json_file:
      json.dump(json_list, json_file, indent=4)

    return

In [ ]:
# 부족한 클래스마다 obj를 때어서 저장해준다. 이것으로 기존 이미지에 붙일계획
data_test_path1 = './data/stencil/Clothing'
data_test_path2 = './data/stencil/Battery'
data_test_path3 = './data/stencil/Glass'
data_test_path4 = './data/stencil/Metal'
data_test_path5 = './data/stencil/PaperPack'
data_test_path6 = './data/stencil/UNKNOWN'
Get_Anno_instance_2(data_test_path1)
Get_Anno_instance_2(data_test_path2)
Get_Anno_instance_2(data_test_path3)
Get_Anno_instance_2(data_test_path4)
Get_Anno_instance_2(data_test_path5)

# 분리한 객체와 annotation을 이용해서 기존 이미지에 적당히 붙여서 부족한 이미지를 추가해 준다.

In [ ]:
# 랜덤 값이 필요하므로 임포트
import random

# 기존 코코 데이터를 읽어와서 이어서 이미지를 추가하고 해당이미지의 annotation을 다시한번 추가하고 추가해준 객체의 annotation도 추가해서 반환해 주는 클래스
class CoCoDataAugment_C(Dataset): # _C는 create 의 첫글자...
    """COCO format"""
    def __init__(self, data_dir, coco_data, train_json_data, aug_data, output_data, out_dir):
        super().__init__()
        #매번 코코 데이터 셋을 읽으면 테스트 하다보면 메모리가 남아나지 않아서 바깥으로부터 읽어와서 변수에 담아놓아서 매번 읽어오지 않는 방식으로 변경.
        self.coco = coco_data

        # 클래스
        self.cat_ids = self.coco.getCatIds()    
        self.cats = self.coco.loadCats(self.cat_ids)

        # 위에서 분리한 객체 데이터도 바깥에서 받아온다.
        self.aug_data = aug_data
        with open(aug_data, "r") as st_json:
          self.aug_data_list = json.load(st_json)

        # 랜덤으로 어떤 객체를 추가할지 정하기 위해서 최대 갯수를 저장
        self.aug_data_count = len(self.aug_data_list)

        # 코코 데이터셋에서 쓰는 함수가 없어서 그냥 json을 가져온다.
        self.train_json = train_json_data
        
        # 이미지를 이어서 쓰기위해서 이미지 개수 가져온다
        self.train_count = len( self.train_json['images'])
        
        # 출력 위치
        self.out_dir = out_dir
        
        # 랜덤한 이미지를 결정하기위해서 이미지 아이디를 가져와서 섞어준다. 어떤이미지를 선택할지 좀더 개선의 여지가 있을듯
        self.train_out_list = list(range(0,self.train_count))
        random.shuffle(self.train_out_list)
        
        # 연속적인 이미지와 annotation id를 부여하기 위해서 저장.
        with open(output_data, "r") as st_json:
          self.output_data = json.load(st_json)
        self.new_img_id =self.output_data['images'][len( self.output_data['images'])-1]["id"]+1
        self.new_anno_id = self.output_data['annotations'][len( self.output_data['annotations'])-1]["id"]+1
        print("new_anno_id",self.new_anno_id)
        print("new_img_id",self.new_img_id)

    def __getitem__(self, index: int):

        # 랜덤으로 추가할 데이터 결정
        aug_id = random.randrange(0,self.aug_data_count)

        # 랜덤으로 원래 그림들 결정
        bg_id = self.train_out_list[index]

        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=bg_id)
        image_infos = self.coco.loadImgs(image_id)[0]

        # cv2 를 활용하여 bg image 불러오기
        bg_image = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        bg_image = cv2.cvtColor(bg_image, cv2.COLOR_BGR2RGB).astype(np.float32)
        bg_image /= 255.0

        # 이미지에 대한 annotation
        anns = self.read_ann_in_img(image_infos['id'])

        # 새로 작성할 json파일을 초기화
        new_image_info = copy.deepcopy(image_infos)  # {"license": 0, "url": null, "file_name": "batch_03/0995.jpg", "height": 512, "width": 512, "date_captured": null, "id": 654}
        new_ann_info = {"id": 0, "image_id": 0, "category_id": 0,"segmentation": [], "area": 0, "bbox": [0,0,0,0], "iscrowd": 0}
        new_img_id = self.new_img_id  # 새로운 이미지 아이디를 얻어오고
        self.new_img_id += 1  # 다음에 추가될 이미지를 위해서 하나 증가 시킨다.
        
        new_image_info["id"] = new_img_id

        # 붙일 이미지와 마스크를 가져온다.
        add_image_path = self.aug_data_list[aug_id]['Image_name']
        #print("add_image_path: ",add_image_path)
        add_mask_path = self.aug_data_list[aug_id]['mask_name']

        #  기존 데이터 형식에서 경로 저장 방식이 변경해서. 경로를 잡아주기 위한 급하게 넣은 코드. -> 내가 데이터 셋을 설계부터하면 이런 의존성을 제거 할수 있을듯.
        # ================================================ 
        # ./data/stencil/Battery/000000044.png
        path1 = add_image_path[:14]
        path2 = add_image_path[14:]

        add_image_path = path1 + "/all" + path2

        path1 = add_mask_path[:14]
        path2 = add_mask_path[14:]

        add_mask_path = path1 + "/all" + path2
        # ./data/stencil/all/Battery/000000044.png
        # ================================================

        # 추가할 이미지 객체 읽어 온다.
        add_image = cv2.imread(add_image_path)
        add_mask = cv2.imread(add_mask_path)

        # 새로 작성할 데이터 생성 area는 bbox를 기준으로 다시 설정한다. id 나중에 추가할 때 길이 값을 이용한다. 
        bg_img_w = new_image_info["width"]  # 이미지 크기
        bg_img_h = new_image_info["height"]  # 이미지 크기

        add_img_w = add_image.shape[1]  # 붙일 이미지 크기
        add_img_h = add_image.shape[0]  # 붙일 이미지 크기

        add_img_area = add_img_w * add_img_h  # annotation의 area
        add_img_cat = self.aug_data_list[aug_id]['label']  # annotation 의 category

        new_anns = copy.deepcopy(anns)  # 기존 이미지의 annotation

        # 기존 이미지의 annotation객체의 area를 기반으로 붙일 이미지가 기존 이미지를 너무 많이 가리지 않기 위해서 평균값을 이용. -> 이부분도 좀더 개선의 여지가 있다.
        anno_cnt = 1
        area_sum = add_img_area
        for i in range(len(new_anns)):
          new_anns[i]["image_id"] = new_img_id  # 이미지 아이디 지정
          new_anns[i]["id"] = self.new_anno_id  # annotation 아이디 지정
          self.new_anno_id += 1  # 다음을 위해서 annotation 아이디 증가
          new_anns[i]['area'] = new_anns[i]['bbox'][2] * new_anns[i]['bbox'][3]  # 기존 이미지 annotation의 bbox 다시 계산
          anno_cnt += 1
          area_sum += new_anns[i]['area']

        ave_area = area_sum/anno_cnt


        # 현재 이미지에 존재하는 annotation의 평균 area와 추가할 image의 area의 비율
        size_ratio = add_img_area / ave_area
        
        # 만약에 추가할 이미지 크기가 크다면? 줄여야 하므로 역수
        if size_ratio >= 1:
          size_ratio = 1/size_ratio

        # 해당 비율로 붙일 사이즈와 이미지를 리사이즈 한다.
        add_image = cv2.cvtColor(add_image, cv2.COLOR_BGR2RGB).astype(np.float32) 
        add_image /= 255.0
        add_mask = cv2.cvtColor(add_mask, cv2.COLOR_BGR2RGB).astype(np.float32)
        add_mask /= 255.0

        # 기존에는 add_mask = cv2.resize(add_mask, dsize=(0, 0), fx=size_ratio, fy=size_ratio, interpolation=cv2.INTER_LINEAR) 방식으로 크리를 줄였는데 중간에 오류가 생긴다... 다른 방식으로 진행한다.
        add_image, add_mask = self.resize_img(add_image, add_mask, size_ratio) 

        # 리사이즈 후 다시 크기 얻어온다.
        add_img_w = add_image.shape[1]  # 붙일 이미지 크기
        add_img_h = add_image.shape[0]  # 붙일 이미지 크기

        add_img_x = random.randrange(0,bg_img_w-add_img_w)  # 붙일 위치
        add_img_y = random.randrange(0,bg_img_h-add_img_h)  # 붙일 위치

        add_img_area = add_img_w * add_img_h  # annotation의 area

        #3차원 텐서를 조작하는 함수를 찾을수 없어서... 각 채널별로 분리해서 마스크를 이용해 이미지를 추가하고 다시 체널을 합친다.
        bg_image_r,bg_image_g,bg_image_b = cv2.split(bg_image)
        add_image_r,add_image_g,add_image_b = cv2.split(add_image)
        add_mask_r,add_mask_g,add_mask_b = cv2.split(add_mask)

        bg_image_r = torch.tensor(bg_image_r)
        bg_image_g = torch.tensor(bg_image_g)
        bg_image_b = torch.tensor(bg_image_b)
        add_mask_r = torch.tensor(add_mask_r)
        add_mask_g = torch.tensor(add_mask_g)
        add_mask_b = torch.tensor(add_mask_b)
        add_image_r = torch.tensor(add_image_r)
        add_image_g = torch.tensor(add_image_g)
        add_image_b = torch.tensor(add_image_b)


        # 기존 이미지에 마스크를 이용해서 이미지를 기존 이미지에 구멍을 뚫는다.
        punched_img_r, _ = self.rm_obj_from_img(bg_image_r,add_mask_r,add_img_x,add_img_y)
        punched_img_g, _ = self.rm_obj_from_img(bg_image_g,add_mask_g,add_img_x,add_img_y)
        punched_img_b, _ = self.rm_obj_from_img(bg_image_b,add_mask_b,add_img_x,add_img_y)
        
        punched_img_ = cv2.merge((punched_img_r.numpy(), punched_img_g.numpy(), punched_img_b.numpy()))
        add_image_ = cv2.merge((add_image_r.numpy(), add_image_g.numpy(), add_image_b.numpy()))        

        # 구멍뚫린 이미지에 이미지를 더한다.
        added_img_r = self.add_obj_to_img(punched_img_r, add_image_r, add_img_x,add_img_y)
        added_img_g = self.add_obj_to_img(punched_img_g, add_image_g, add_img_x,add_img_y)
        added_img_b = self.add_obj_to_img(punched_img_b, add_image_b, add_img_x,add_img_y)

        added_img_b = added_img_b.numpy()
        added_img_g = added_img_g.numpy()
        added_img_r = added_img_r.numpy()

        # 다시 채널별로 합친다.
        added_img = cv2.merge((added_img_r, added_img_g, added_img_b))
        added_img_bgr = cv2.merge((added_img_b, added_img_g, added_img_r))
        added_img_bgr *= 225.0

        # self.train_json(기존 annotation)에 추가된 이미지의 annotation을 추가한다.
        new_ann_info["id"] = self.new_anno_id
        self.new_anno_id += 1
        new_ann_info["image_id"] = new_img_id
        new_ann_info["category_id"] = add_img_cat
        new_ann_info["bbox"][0] =  add_img_x
        new_ann_info["bbox"][1] =  add_img_y
        new_ann_info["bbox"][2] =  add_image.shape[1]
        new_ann_info["bbox"][3] =  add_image.shape[0]
        new_ann_info["area"] = add_image.shape[1]*add_image.shape[0]
        new_anns.append(new_ann_info)

        new_img_path = os.path.join(self.out_dir,f'{new_img_id:0{9}}.jpg')  # 새로 저장할 이미지 path
        new_image_info['file_name'] = new_img_path

        return added_img_bgr, new_anns, new_image_info

    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

    # 이미지와 마스크 이미지에서 bbox만큼 크롭한다.
    def crop_with_bbox(self, image, mask, bbox):
        x_min = bbox[0]
        y_min = bbox[1]
        x_max = bbox[0] + bbox[2]
        y_max = bbox[1] + bbox[3]
        transform = A.Compose([
                    A.Crop(int(x_min),int(y_min),int(x_max),int(y_max)),
                    ToTensorV2()
            ])
        transformed = transform(image=image, mask=mask)
        crop_img = transformed["image"]
        #print("crop_img type: ",type(crop_img))
        crop_masks = transformed["mask"]
        #print("crop_masks type: ",type(crop_masks))
        return crop_img, crop_masks

    # 마스크를 이용해서 이미지에서 오브젝트만 남기고 지운다. 파일에서 보기 좋게 알파값도 만들어 온다.
    def rm_bg_from_img(self, image, mask):
        bit_mask_base = torch.ones_like(image).to(torch.int8)
        alpha = torch.ones_like(mask).to(torch.float)

        for i in list(torch.unique(mask)):
          if i == 0:
            continue
          bit_mask = bit_mask_base * i
          bit_mask = torch.eq(mask, bit_mask)                    
          temp_image =  image * bit_mask
          temp_alpha = alpha * bit_mask[0]
        return  temp_image, temp_alpha
    
    # 마스크를 이용해서 이미지에서 오브젝트만 지운다. 파일에서 보기 좋게 알파값도 만들어 온다.
    def rm_obj_from_img(self, image, mask,x,y):
        # 알파 값을 기록할 텐서
        alpha = torch.ones_like(image).to(torch.float)
        # 값이 0인것만 뽑아낼 기준
        bit_mask_base = torch.zeros_like(image).to(torch.int8)

        # 잘린 마스크를 x,y 위치에 붙이는 작업
        img_w = image.shape[1]  # 이미지 크기
        img_h = image.shape[0]  # 이미지 크기
        mask_w = mask.shape[1]  # 마스크 크기
        mask_h = mask.shape[0]  # 마스크 크기
        # 마스크의 크기와 이미지의 크기가 다르므로 직사각형을 더해서 이미지 크기로 만들어 준다
        chunk1 = torch.zeros(mask_h,x)
        chunk2 = torch.zeros(y, x + mask_w)
        chunk3 = torch.zeros(mask_h + y,img_w - (x + mask_w))
        chunk4 = torch.zeros(img_h - (mask_h + y),img_w)

        mask = torch.cat([chunk1,mask],dim=1)
        mask = torch.cat([chunk2,mask],dim=0)
        mask = torch.cat([mask,chunk3],dim=1)
        mask = torch.cat([mask,chunk4],dim=0)
        #print("mask",mask.shape)
        # 값이 0인 객체가 아닌 부분만 True(1) 나머지는 False로 만든다.
        inverse_bit_mask = torch.eq(mask, bit_mask_base)
        # 이미지에 elemwise로 곱해서 객체부분만 0으로 만듬
        temp_image = image*inverse_bit_mask
        # 알파 값도 객체 부분은 0으로 투명하게

        temp_alpha = alpha * inverse_bit_mask
        return  temp_image, temp_alpha

    # 이미지에 오브젝트를 추가 한다.
    def add_obj_to_img(self,image, add_img, x, y):
        img_w = image.shape[1]  # 이미지 크기
        img_h = image.shape[0]  # 이미지 크기
        add_img_w = add_img.shape[1]  # 마스크 크기
        add_img_h = add_img.shape[0]  # 마스크 크기
        
        # 마스크의 크기와 이미지의 크기가 다르므로 직사각형을 더해서 이미지 크기로 만들어 준다
        chunk1 = torch.zeros(add_img_h, x)
        chunk2 = torch.zeros(y, x + add_img_w)
        chunk3 = torch.zeros(add_img_h + y, img_w - (x + add_img_w))
        chunk4 = torch.zeros(img_h - (add_img_h + y),img_w)
        add_img = torch.cat([chunk1,add_img],dim=1)
        add_img = torch.cat([chunk2,add_img],dim=0)
        add_img = torch.cat([add_img,chunk3],dim=1)
        add_img = torch.cat([add_img,chunk4],dim=0)

        temp_image = add_img + image
        return  temp_image
    
    # 이미지 크기를 변경한다.
    def resize_img(self, image, mask, ratio):
        img_w = image.shape[1]  # 붙일 이미지 크기
        img_h = image.shape[0]  # 붙일 이미지 크기
        w = int(img_w*ratio)
        h = int(img_h*ratio)

        #너무 크거나 줄어들면 오류 발생 적당한 최대 최소 값을 지장.
        if(w>=256) or (w < 25):
          w = 80
          h = 80
        if(h>=256) or (h < 25):
          w = 80
          h = 80

        transform = A.Compose([
                    A.RandomRotate90(),
                    A.HorizontalFlip(p=0.5),
                    A.VerticalFlip(p=0.5),
                    A.Resize(width=w,height=h,p=1.0)
            ])
        transformed = transform(image=image, mask=mask)
        _img = transformed["image"]

        _masks = transformed["mask"]

        return _img, _masks

    def read_ann_in_img(self, img_id):
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        return anns

# 생성된 새로운 이미지, annotation을 이용해서 기존 coco데이터를 수정후 저장 하는 함수

In [ ]:
def Get_Anno_instance_3(data_dir, aug_data, output_cnt, out_dir, output_data,coco_data,train_json_data):
  
  train_dataset = CoCoDataAugment_C(data_dir=data_dir, coco_data=coco_data, train_json_data=train_json_data, aug_data=aug_data, output_data=output_data, out_dir=out_dir)
  data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=1,
                                           shuffle=True,
                                           num_workers=1,
                                           collate_fn=collate_fn,
                                           drop_last=True)
  cat_names = ['UNKNOWN','General trash','Paper','Paper pack','Metal','Glass','Plastic','Styrofoam','Plastic bag','Battery','Clothing']

  new_anns_list = []
  new_image_info_list = []

  for step,(added_img, new_anns, new_image_info) in tqdm(enumerate(data_loader)):
    if step > output_cnt:
      break
    
    boxes = []
    cats = []
    for i in range(len(new_anns[0])):
      new_anns_list.append(new_anns[0][i])
      boxes.append(new_anns[0][i]["bbox"])
      cats.append(new_anns[0][i]["category_id"])

    new_image_info_list.append(new_image_info[0])
    cv2.imwrite(new_image_info[0]["file_name"], added_img[0])

    # visualize(added_img[0]/255, boxes, cats, cat_names)
    
  image_dict = {"images" : new_image_info_list}
  anno_dict = {"annotation" : new_anns_list}

  with open(output_data,"r") as out_json_file:
    out_json_data = json.load(out_json_file)
  
  print("\nimage id last: ", out_json_data["images"][len(out_json_data["images"])-1]['id'])
  img_s_idx = len(out_json_data["images"])
                  
  for image_info in new_image_info_list:
    out_json_data["images"].append(image_info)
  
  print("image id start: ", out_json_data["images"][img_s_idx]['id'])
  print("image id end: ", out_json_data["images"][len(out_json_data["images"])-1]['id'])
  
  print("anno id last: ", out_json_data["annotations"][len(out_json_data["annotations"])-1]['id'])
  anno_s_idx = len(out_json_data["annotations"])
  for anno_info in new_anns_list:
    out_json_data["annotations"].append(anno_info)

  print("anno id start: ", out_json_data["annotations"][anno_s_idx]['id'])
  print("anno id end: ", out_json_data["annotations"][len(out_json_data["annotations"])-1]['id'])

  with open(output_data, 'w') as json_file:
    json.dump(out_json_data, json_file, indent=4)
  
  imgjdir = "./data" + out_dir + '/image_infos.json'
  annojdir = "./data" + out_dir + '/anno_infos.json'
  

  with open(imgjdir, 'w') as json_file:
    json.dump(image_dict, json_file, indent=4)
  
  with open(annojdir, 'w') as json_file:
    json.dump(anno_dict, json_file, indent=4)
  

# 각 배터리, 옷, 유리, 철, 종이팩이 추가된 데이터를 만든다.

In [ ]:
coco = COCO(anns_file_path)
with open(anns_file_path, "r") as st_json:
  train_json = json.load(st_json)

Get_Anno_instance_3(data_dir=anns_file_path, aug_data='./data/stencil/all/Battery/stencil.json', output_cnt=500, out_dir='/Battery',output_data='./data/new_train_all.json', coco_data=coco, train_json_data=train_json)#,coco_data,train_json_data
Get_Anno_instance_3(data_dir=anns_file_path, aug_data='./data/stencil/all/Clothing/stencil.json', output_cnt=500, out_dir='/Clothing',output_data='./data/new_train_all.json', coco_data=coco, train_json_data=train_json)
Get_Anno_instance_3(data_dir=anns_file_path, aug_data='./data/stencil/all/Glass/stencil.json', output_cnt=500, out_dir='/Glass',output_data='./data/new_train_all.json', coco_data=coco, train_json_data=train_json)
Get_Anno_instance_3(data_dir=anns_file_path, aug_data='./data/stencil/all/Metal/stencil.json', output_cnt=500, out_dir='/Metal',output_data='./data/new_train_all.json', coco_data=coco, train_json_data=train_json)
Get_Anno_instance_3(data_dir=anns_file_path, aug_data='./data/stencil/all/PaperPack/stencil.json', output_cnt=500, out_dir='/PaperPack',output_data='./data/new_train_all.json', coco_data=coco, train_json_data=train_json)